In [1]:
import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
## Data Import
import dgl.data

dataset = dgl.data.CoraGraphDataset()
print("Number of Categories: ", dataset.num_classes)

  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.
Number of Categories:  7


In [3]:
## Dataset has only one graph, although it can have multiple
g = dataset[0]

A DGL graph can store node features and edge features in two dictionary-like attributes called ndata and edata

In [4]:
print("Node Features")
print(g.ndata)
print("Shape of Node Features: ", g.ndata['feat'].shape)
print("Edge Features")
print(g.edata)

Node Features
{'feat': tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]]), 'label': tensor([3, 4, 4,  ..., 3, 3, 3]), 'test_mask': tensor([False, False, False,  ...,  True,  True,  True]), 'train_mask': tensor([ True,  True,  True,  ..., False, False, False]), 'val_mask': tensor([False, False, False,  ..., False, False, False])}
Shape of Node Features:  torch.Size([2708, 1433])
Edge Features
{}


In [5]:
## Building a 2 layer Graph Convolution Network
from dgl.nn import GraphConv

class GCN(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GraphConv(in_feats, h_feats)
        self.conv2 = GraphConv(h_feats, num_classes)
        
    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        
        return h
    
## Creating the model for our example
model = GCN(g.ndata['feat'].shape[1], 16, dataset.num_classes)

In [6]:
## Training the GCN

def train(g, model):
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.01)
    best_val_acc = 0
    best_test_acc = 0
    
    features = g.ndata['feat']
    labels = g.ndata['label']
    train_mask = g.ndata['train_mask']
    val_mask = g.ndata['val_mask']
    test_mask = g.ndata['test_mask']
    
    for e in range(100):
        logits = model(g, features)
        
        pred = logits.argmax(1)
        
        loss = F.cross_entropy(logits[train_mask], labels[train_mask])
        
        train_acc = (pred[train_mask] == labels[train_mask]).float().mean()
        val_acc = (pred[val_mask] == labels[val_mask]).float().mean()
        test_acc = (pred[test_mask] == labels[test_mask]).float().mean()
        
        if best_val_acc < val_acc:
            best_val_acc = val_acc
            best_test_acc = test_acc
            
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if e%5 == 0:
            print('In epoch {}, loss: {:.3f}, val acc: {:.3f} (best {:.3f}), test acc: {:.3f} (best {:.3f})'.format(
                e, loss, val_acc, best_val_acc, test_acc, best_test_acc))

In [7]:
torch.__version__

'1.12.1+cu102'

In [9]:
train(g, model)

In epoch 0, loss: 1.946, val acc: 0.176 (best 0.176), test acc: 0.192 (best 0.192)
In epoch 5, loss: 1.904, val acc: 0.544 (best 0.544), test acc: 0.572 (best 0.572)
In epoch 10, loss: 1.833, val acc: 0.664 (best 0.670), test acc: 0.689 (best 0.691)
In epoch 15, loss: 1.742, val acc: 0.686 (best 0.694), test acc: 0.705 (best 0.706)
In epoch 20, loss: 1.628, val acc: 0.688 (best 0.694), test acc: 0.709 (best 0.706)
In epoch 25, loss: 1.494, val acc: 0.700 (best 0.700), test acc: 0.702 (best 0.703)
In epoch 30, loss: 1.340, val acc: 0.704 (best 0.704), test acc: 0.709 (best 0.709)
In epoch 35, loss: 1.174, val acc: 0.696 (best 0.704), test acc: 0.711 (best 0.709)
In epoch 40, loss: 1.004, val acc: 0.706 (best 0.706), test acc: 0.719 (best 0.719)
In epoch 45, loss: 0.842, val acc: 0.704 (best 0.708), test acc: 0.717 (best 0.718)
In epoch 50, loss: 0.694, val acc: 0.718 (best 0.718), test acc: 0.722 (best 0.722)
In epoch 55, loss: 0.565, val acc: 0.718 (best 0.718), test acc: 0.723 (best 0